<a href="https://colab.research.google.com/github/mitul01/FER/blob/main/EmotionNet_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/colab-20210303T131708Z-001/colab

/content/drive/MyDrive/colab-20210303T131708Z-001/colab


# EmotionNet VGG

In [3]:
# set matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import packages
from config import emotion_config as config
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
from pipeline.io import HDF5DatasetGenerator
from pipeline.nn.conv import EmotionVGGNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import load_model
from keras import backend as K
import argparse
import os

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score


# construct the training and testing image generators for data
# augmentation, then initialize the image preprocessor
trainAug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.1,
    horizontal_flip = True, rescale = 1 / 255.0, fill_mode = "nearest")

valAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
#if args["model"] is None:
print("compiling model...")
model = EmotionVGGNet.build(width = 48, height = 48, depth = 1,
    classes = config.NUM_CLASSES)
# opt = SGD(lr = 1e-2, momentum = 0.9, nesterov = True)
opt = Adam(lr = 1e-3)
model.compile(loss = "categorical_crossentropy", optimizer = opt,
    metrics = ["accuracy"])

# otherwise, load the checkpoint from disk
#else:
#    print("[INFO] loding {}...".format(args["model"]))
#    model = load_model(args["model"])

# update the learning rate
# print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
# K.set_value(model.optimizer.lr, 1e-5)
# print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

# construct the set of callbacks
figPath = os.path.sep.join([config.OUTPUT_PATH, "vggnet_emotion_1.png"])
jsonPath = os.path.sep.join([config.OUTPUT_PATH, "vggnet_emotion_1.json"])

callbacks = [
    EpochCheckpoint("checkpoints", every = 5, #startAt = args["start_epoch"]
                     ),TrainingMonitor(figPath, jsonPath = jsonPath) 
                     #startAt = args["start_epoch"]och"]                    
 ]

# lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3)
# early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
# checkpointer = ModelCheckpoint('drive/My Drive/weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=7,
    verbose=1,
    restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.6,
    patience=4,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

# train network

history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // config.BATCH_SIZE,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // config.BATCH_SIZE,
    epochs = 100,
    max_queue_size = config.BATCH_SIZE * 2,
    callbacks = callbacks,
    verbose = 1
    
)

# close the dataset

trainGen.close()
valGen.close()

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


compiling model...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
224/224 [==============================] - 48s 63ms/step - loss: 3.5553 - accuracy: 0.2071 - val_loss: 2.8527 - val_accuracy: 0.2955
Epoch 2/100
224/224 [==============================] - 13s 58ms/step - loss: 2.7609 - accuracy: 0.3228 - val_loss: 2.3620 - val_accuracy: 0.4478
Epoch 3/100
224/224 [==============================] - 13s 57ms/step - loss: 2.4031 - accuracy: 0.4013 - val_loss: 2.1451 - val_accuracy: 0.4791
Epoch 4/100
224/224 [==============================] - 13s 56ms/step - loss: 2.1399 - accuracy: 0.4600 - val_loss: 1.8961 - val_accuracy: 0.5265
Epoch 5/100
224/224 [==============================] - 13s 57ms/step - loss: 1.9514 - accuracy: 0.4919 - val_loss: 1.7162 - val_accuracy: 0.5550
Epoch 6/100
224/224 [==============================] - 13s 57ms/step - loss: 1.8047 - accuracy: 0.5133 - val_loss: 1.6166 - val_accuracy: 0.5552
Epoch 7/100
224/224 [==============================] - 13s 58ms/step - loss: 1.6908 - accuracy: 0.5291 - val_loss: 1.5236 - val_ac

In [4]:
testAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

testGen = HDF5DatasetGenerator(config.TEST_HDF5, config.BATCH_SIZE,
    aug = testAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [5]:
predictions = model.predict_generator(testGen.generator(),steps=testGen.numImages //64, max_queue_size=64*2)

(loss, acc) = model.evaluate_generator(
	testGen.generator(),
	steps=testGen.numImages // config.BATCH_SIZE,
	max_queue_size=config.BATCH_SIZE * 2)
print("[INFO] accuracy: {:.2f}".format(acc * 100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[INFO] accuracy: 66.27


#SeNet

In [ ]:
!pip install keras-vggface
!pip install keras_applications

In [ ]:
from keras_vggface.vggface import VGGFace
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline
from config import emotion_config as config
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.io import HDF5DatasetGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from keras.optimizers import Adam
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
import os

In [ ]:
trainAug = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)

valAug= ImageDataGenerator(rescale=1./255,)

iap = ImageToArrayPreprocessor()

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES,)

valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/My Drive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [ ]:
EPOCHS = 100
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 201

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197
vgg_notop = VGGFace(input_shape=(197,197,3),model='senet50',include_top=False)
last_layer = vgg_notop.get_layer('avg_pool').output

x = Flatten(name='flatten')(last_layer)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(0.5)(x)

batch_norm_indices = [2, 6, 9, 12, 21, 25, 28, 31, 42, 45, 48, 59, 62, 65, 74, 78, 81, 84, 95, 98, 101, 112, 115, 118, 129, 132, 135, 144, 148, 151, 154, 165, 168, 171, 182, 185, 188, 199, 202, 205, 216, 219, 222, 233, 236, 239, 248, 252, 255, 258, 269, 272, 275]    
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False

out = Dense(6, activation='softmax', name='classifier')(x)

model = Model(vgg_notop.input, out)

In [ ]:
opt = Adam(lr = 1e-4)
model.compile(loss = "categorical_crossentropy", optimizer = opt,metrics = ["accuracy"])

figPath = os.path.sep.join([config.OUTPUT_PATH, "vggface_senet50.png"])
jsonPath = os.path.sep.join([config.OUTPUT_PATH, "vggface_senet50.json"])

callbacks = [
    EpochCheckpoint("checkpoints", every = 5, #startAt = args["start_epoch"]
                     ),TrainingMonitor(figPath, jsonPath = jsonPath) 
                     #startAt = args["start_epoch"]och"]                    
 ]

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

# train network
history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // config.BATCH_SIZE,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // config.BATCH_SIZE,
    epochs = 100,
    max_queue_size = config.BATCH_SIZE * 2,
    callbacks = callbacks,
    verbose = 1
    
)

# close the dataset

trainGen.close()
valGen.close()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100


InvalidArgumentError: ignored

In [ ]:
!pip install keras-vggface
!pip install keras_applications

     |████████████████████████████████| 51kB 6.4MB/s 


In [ ]:
#from keras_vggface.vggface import VGGFace
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline
from config import emotion_config as config
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
#from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.io import HDF5DatasetGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from keras.optimizers import Adam
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
import os

In [ ]:
trainAug = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)

valAug= ImageDataGenerator(rescale=1./255,)

iap = ImageToArrayPreprocessor()

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES,)

valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

In [ ]:
model = VGGFace(classes = config.NUM_CLASSES,weights=None,input_shape=(48,48,1))

opt = Adam(lr = 1e-4)
model.compile(loss = "categorical_crossentropy", optimizer = opt,metrics = ["accuracy"])

figPath = os.path.sep.join([config.OUTPUT_PATH, "vggface.png"])
jsonPath = os.path.sep.join([config.OUTPUT_PATH, "vggface.json"])

callbacks = [
    EpochCheckpoint("checkpoints", every = 5, #startAt = args["start_epoch"]
                     ),TrainingMonitor(figPath, jsonPath = jsonPath) 
                     #startAt = args["start_epoch"]och"]                    
 ]

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

# train network

history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // config.BATCH_SIZE,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // config.BATCH_SIZE,
    epochs = 100,
    max_queue_size = config.BATCH_SIZE * 2,
    callbacks = callbacks,
    verbose = 1
    
)

# close the dataset

trainGen.close()
valGen.close()

In [ ]:
model.save_weights('model_vggFace.h5')